In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [2]:
df=pd.read_csv('Salary_Survey.csv')

EDA process

In [4]:
df.head()

Timestamp How old are you?  What industry do you work in?  \
0  4/27/2021 11:02:10            25-34   Education (Higher Education)   
1  4/27/2021 11:02:22            25-34              Computing or Tech   
2  4/27/2021 11:02:38            25-34  Accounting, Banking & Finance   
3  4/27/2021 11:02:41            25-34                     Nonprofits   
4  4/27/2021 11:02:42            25-34  Accounting, Banking & Finance   

                                  Job title  \
0        Research and Instruction Librarian   
1  Change & Internal Communications Manager   
2                      Marketing Specialist   
3                           Program Manager   
4                        Accounting Manager   

  If your job title needs additional context, please clarify here:  \
0                                                NaN                 
1                                                NaN                 
2                                                NaN                 
3                                                NaN                 
4                                                NaN                 

  What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)  \
0                                             55,000                                                                                                                                                                                     
1                                             54,600                                                                                                                                                                                     
2                                             34,000                                                                                                                                                                                     
3                                             62,000                                                                                                                                                                                     
4                                             60,000                                                                                                                                                                                     

   How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.  \
0                                                0.0                                                                                                                                                
1                                             4000.0                                                                                                                                                
2                                                NaN                                                                                                                                                
3                                             3000.0                                                                                                                                                
4                                             7000.0                                                                                                                                                

  Please indicate the currency  \
0                          USD   
1                          GBP   
2                          USD   
3                          USD   
4                          USD   

  If "Other," please indicate the currency here:   \
0                      

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28178 entries, 0 to 28177
Data columns (total 23 columns):
 #   Column                                                                                                                                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                --------------  -----  
 0   Timestamp                                                                                                                                                                                                                             28085 non-null  object 
 1   How old are you?                                                                                             

In [6]:
# Dropping the features with no values
df=df.drop(columns=['Timestamp', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22'])

In [7]:
# Dropping oservations with no values
df.dropna(axis=0, how='all', inplace=True)

In [8]:
# Simplifying features names
df.columns = ['Age','Industry','Job_title','Additional_job_details','Annual_salary','Monetary_compensation','Currency','Other_Currency','Additional_income_details','Country_of_work','U.S._state_of_work','City_of_work','Years_of_experience','Years_of_experience_in_your_field','Level_of_education','Gender']

In [9]:
df.head()

Age                       Industry  \
0  25-34   Education (Higher Education)   
1  25-34              Computing or Tech   
2  25-34  Accounting, Banking & Finance   
3  25-34                     Nonprofits   
4  25-34  Accounting, Banking & Finance   

                                  Job_title Additional_job_details  \
0        Research and Instruction Librarian                    NaN   
1  Change & Internal Communications Manager                    NaN   
2                      Marketing Specialist                    NaN   
3                           Program Manager                    NaN   
4                        Accounting Manager                    NaN   

  Annual_salary  Monetary_compensation Currency Other_Currency  \
0        55,000                    0.0      USD            NaN   
1        54,600                 4000.0      GBP            NaN   
2        34,000                    NaN      USD            NaN   
3        62,000                 3000.0      USD            NaN   
4        60,000                 7000.0      USD            NaN   

  Additional_income_details Country_of_work U.S._state_of_work City_of_work  \
0                       NaN   United States      Massachusetts       Boston   
1                       NaN  United Kingdom                NaN    Cambridge   
2                       NaN              US          Tennessee  Chattanooga   
3                       NaN             USA          Wisconsin    Milwaukee   
4                       NaN              US     South Carolina   Greenville   

  Years_of_experience Years_of_experience_in_your_field Level_of_education  \
0           5-7 years                         5-7 years    Master's degree   
1        8 - 10 years                         5-7 years     College degree   
2         2 - 4 years                       2 - 4 years     College degree   
3        8 - 10 years                         5-7 years     College degree   
4        8 - 10 years                         5-7 years     College degree   

       Gender  
0       Woman  
1  Non-binary  
2       Woman  
3       Woman  
4       Woman

In [10]:
# Dropping features with no relevant data to my final analysis
reduced_df = df.drop(columns=['Additional_job_details', 'Monetary_compensation', 'Other_Currency', 'Additional_income_details', 'U.S._state_of_work', 'City_of_work'])

In [11]:
# Count of the missing values
reduced_df.isnull().sum()

Age                                    0
Industry                              74
Job_title                              1
Annual_salary                          0
Currency                               0
Country_of_work                        0
Years_of_experience                    0
Years_of_experience_in_your_field      0
Level_of_education                   222
Gender                               171
dtype: int64

CLEANING THE DATA

In [13]:
# Checking data types
# Object is categorical data and some features in this dataset should be numerical data
reduced_df.dtypes

Age                                  object
Industry                             object
Job_title                            object
Annual_salary                        object
Currency                             object
Country_of_work                      object
Years_of_experience                  object
Years_of_experience_in_your_field    object
Level_of_education                   object
Gender                               object
dtype: object

In [14]:
# Checking for malformed values in all of the features
reduced_df["Age"].unique()

array(['25-34', '45-54', '35-44', '18-24', '65 or over', '55-64',
       'under 18'], dtype=object)

In [15]:
# Mapping age groups as midpoint numerical values
# Reference; Geeksforgeeks, https://www.geeksforgeeks.org/using-dictionary-to-remap-values-in-pandas-dataframe-columns/
# Reference: Stack Overflow, https://stackoverflow.com/questions/46432315/pandas-replace-column-values-by-dictionary-keys-if-they-are-in-dictionary-valu
age_category_mapping = {
    "under 18": 17,
    "18-24": 21,
    "25-34": 29,
    "35-44": 39,
    "45-54": 49,
    "55-64": 59,
    "65+": 65
}

In [16]:
reduced_df["Average_age"] = reduced_df["Age"].map(age_category_mapping)

In [17]:
reduced_df.drop(columns=["Age"], inplace=True)

In [18]:
print(reduced_df)

                                   Industry  \
0              Education (Higher Education)   
1                         Computing or Tech   
2             Accounting, Banking & Finance   
3                                Nonprofits   
4             Accounting, Banking & Finance   
...                                     ...   
28080         Accounting, Banking & Finance   
28081         Education (Primary/Secondary)   
28082  Government and Public Administration   
28083                     Computing or Tech   
28084                       Social networks   

                                      Job_title Annual_salary Currency  \
0            Research and Instruction Librarian        55,000      USD   
1      Change & Internal Communications Manager        54,600      GBP   
2                          Marketing Specialist        34,000      USD   
3                               Program Manager        62,000      USD   
4                            Accounting Manager        60,000    

In [19]:
reduced_df.dtypes

Industry                              object
Job_title                             object
Annual_salary                         object
Currency                              object
Country_of_work                       object
Years_of_experience                   object
Years_of_experience_in_your_field     object
Level_of_education                    object
Gender                                object
Average_age                          float64
dtype: object

In [20]:
reduced_df["Gender"].unique()

array(['Woman', 'Non-binary', 'Man', nan, 'Other or prefer not to answer',
       'Prefer not to answer'], dtype=object)

In [21]:
# Mapping other important features like Gender to get cleaner data for final analysis
gender_category_mapping = {
    "Man": "Male",
    "Woman": "Female",
    "Other or prefer not to answer": "Prefer not to answer",
    "Non-binary": "Non-binary",
}

In [22]:
reduced_df["Gender_category"] = reduced_df["Gender"].map(gender_category_mapping)

In [23]:
reduced_df.drop(columns=["Gender"], inplace=True)

In [24]:
print(reduced_df)

                                   Industry  \
0              Education (Higher Education)   
1                         Computing or Tech   
2             Accounting, Banking & Finance   
3                                Nonprofits   
4             Accounting, Banking & Finance   
...                                     ...   
28080         Accounting, Banking & Finance   
28081         Education (Primary/Secondary)   
28082  Government and Public Administration   
28083                     Computing or Tech   
28084                       Social networks   

                                      Job_title Annual_salary Currency  \
0            Research and Instruction Librarian        55,000      USD   
1      Change & Internal Communications Manager        54,600      GBP   
2                          Marketing Specialist        34,000      USD   
3                               Program Manager        62,000      USD   
4                            Accounting Manager        60,000    

In [25]:
reduced_df["Industry"].unique()

array(['Education (Higher Education)', 'Computing or Tech',
       'Accounting, Banking & Finance', ..., 'Student ', 'Wine & Spirits',
       'Social networks'], dtype=object)

In [26]:
reduced_df["Job_title"].unique()

array(['Research and Instruction Librarian',
       'Change & Internal Communications Manager', 'Marketing Specialist',
       ..., 'Curriculum Writer', 'Software Engineering Co-Op',
       'Content creator'], dtype=object)

In [27]:
reduced_df["Years_of_experience"].unique()

array(['5-7 years', '8 - 10 years', '2 - 4 years', '21 - 30 years',
       '11 - 20 years', '1 year or less', '41 years or more',
       '31 - 40 years'], dtype=object)

In [28]:
# Mapping Years_of_experience feature as midpoint values and converting to numerical
years_category_mapping = {
    "1 year or less": "1",
    "2 - 4 years": "3",
    "5-7 years": "6",
    "8 - 10 years": "9",
    "11 - 20 years": "15",
    "21 - 30 years": "25",
    "31 - 40 years": "35",
    "41 years or more": "41",
}

In [29]:
reduced_df["Average_Years_of_experience"] = reduced_df["Years_of_experience"].map(years_category_mapping)

In [30]:
reduced_df.drop(columns=["Years_of_experience"], inplace=True)

In [31]:
print(reduced_df)

                                   Industry  \
0              Education (Higher Education)   
1                         Computing or Tech   
2             Accounting, Banking & Finance   
3                                Nonprofits   
4             Accounting, Banking & Finance   
...                                     ...   
28080         Accounting, Banking & Finance   
28081         Education (Primary/Secondary)   
28082  Government and Public Administration   
28083                     Computing or Tech   
28084                       Social networks   

                                      Job_title Annual_salary Currency  \
0            Research and Instruction Librarian        55,000      USD   
1      Change & Internal Communications Manager        54,600      GBP   
2                          Marketing Specialist        34,000      USD   
3                               Program Manager        62,000      USD   
4                            Accounting Manager        60,000    

In [32]:
reduced_df.dtypes

Industry                              object
Job_title                             object
Annual_salary                         object
Currency                              object
Country_of_work                       object
Years_of_experience_in_your_field     object
Level_of_education                    object
Average_age                          float64
Gender_category                       object
Average_Years_of_experience           object
dtype: object

In [33]:
# Reference: Stack Overflow https://stackoverflow.com/questions/48094854/pandas-convert-data-type-from-object-to-float
# Converting to numerical values
reduced_df["Average_Years_of_experience"] = reduced_df["Average_Years_of_experience"].astype(float)

In [34]:
reduced_df.dtypes

Industry                              object
Job_title                             object
Annual_salary                         object
Currency                              object
Country_of_work                       object
Years_of_experience_in_your_field     object
Level_of_education                    object
Average_age                          float64
Gender_category                       object
Average_Years_of_experience          float64
dtype: object

In [35]:
reduced_df["Level_of_education"].unique()

array(["Master's degree", 'College degree', 'PhD', nan, 'Some college',
       'High School', 'Professional degree (MD, JD, etc.)'], dtype=object)

In [36]:
reduced_df["Country_of_work"].unique()

array(['United States', 'United Kingdom', 'US', 'USA', 'Canada',
       'United Kingdom ', 'usa', 'UK', 'Scotland ', 'U.S.',
       'United States ', 'The Netherlands', 'Australia ', 'Spain', 'us',
       'Usa', 'England', 'finland', 'United States of America', 'France',
       'United states', 'Scotland', 'USA ', 'United states ', 'Germany',
       'UK ', 'united states', 'Ireland', 'India', 'Australia', 'Uk',
       'United States of America ', 'U.S. ', 'canada', 'Canada ', 'U.S>',
       'ISA', 'Argentina', 'Great Britain ', 'US ', 'United State',
       'U.S.A', 'Denmark', 'U.S.A.', 'America', 'Netherlands',
       'netherlands', 'England ', 'united states of america', 'Ireland ',
       'Switzerland', 'Netherlands ', 'Bermuda', 'Us',
       'The United States', 'United State of America', 'Germany ',
       'Malaysia', 'Mexico ', 'United Stated', 'South Africa ', 'Belgium',
       'Northern Ireland', 'u.s.', 'South Africa', 'UNITED STATES',
       'united States', 'Sweden', 'Hong K

In [37]:
# Creating a malformed data and typos list and replacing it with coutry names or acronyms to get cleaner data in feauture Country_of_work
mvp_us=["USA", "United States", "usa", "us", "US", "US ", "United States ", "USA ", "united states", "U.S>", "U.S. ","United States of America ","United State","U.S.A","America","united states of america","Us","The United States","U.S.A.",'United Stated','United State of America','u.s.','UNITED STATES','united States','USA-- Virgin Islands','United Statws','U.S','Unites States ','Usa ','U.S.A. ','U. S. ','United Sates','United States of American ','Uniited States','Worldwide (based in US but short term trips aroudn the world)','United Sates of America','United States (I work from home and my clients are all over the US/Canada/PR','Unted States','United Statesp','United Stattes','United Statea','Unites States','United Statees','Uniyes States','UNited States','I am located in Canada but I work for a company in the US','Uniyed states','Uniyes States','United States of Americas','U. S.','United States of America','Usa','United states ','United states','US of A','United States of america ','U.SA','United Status',' U.S.','Puerto Rico ','Canada and USA','Virginia','U.s.','U.s.a.','USS','Uniteed States','Japan, US Gov position','United Stares','Us ','Unite States','The US','united states ','United states of America ','UnitedStates','UK for U.S. company','For the United States government, but posted overseas',' United States', 'United Statues', 'Untied States',
       'USA (company is based in a US territory, I work remote)','united stated',
       'United States Of America','USAB', 'Unitied States',"I work for an US based company but I'm from Argentina.",'Uniter Statez', 'U. S ','United Sttes','Unitef Stated','United states of America','San Francisco', 'Usat', '🇺🇸 ','United States of america','From Romania, but for an US based company','USA tomorrow ',
       'United Stateds','United states of america', 'UsA','United States of American', 'U.S.A ','USaa', 'uSA','US govt employee overseas, country withheld', 'usa ','uS','United Stares ','United STates','america','California ','United States is America','United States- Puerto Rico','United Statss','United  States','ISA',' US',
       'Unites states ','United statew',]
clean_reduced_df=reduced_df.replace(mvp_us, "U.S.")
# Specifying the column for which I want to change USD so it does not affect Currency feature
clean_reduced_df["Country_of_work"] = clean_reduced_df["Country_of_work"].replace("USD", "U.S.")

In [38]:
mvp_uk=['United Kingdom','United Kingdom ', 'UK', 'Scotland ','England','Scotland','UK ','Uk','Great Britain ','England ','Northern Ireland','England/UK','England, UK.','Britain ','United Kingdom (England)','United Kingdom.','United kingdom','United kingdom ','U.K. ','England, UK','uk','Jersey, Channel islands', 'Uk ','Great Britain','Hartford','UK (Northern Ireland)','United Kindom','Wales (United Kingdom)', 'England, Gb','Wales','Northern Ireland ', 'england','U.K. (northern England)', 'U.K','UK (England)', 'UK, remote',
       'Scotland, UK','Unites kingdom ','United Kingdomk','united kingdom','Austria, but I work remotely for a Dutch/British company','Wales, UK','England, United Kingdom','Englang','England, United Kingdom ','United y',
       'Wales (UK)', 'Isle of Man', 'Northern Ireland, United Kingdom','ENGLAND','London',]
clean_reduced_df=clean_reduced_df.replace(mvp_uk, "U.K.")

In [39]:
mvp_ca=['canada', 'Canada ','Canada, Ottawa, ontario','Can','CANADA ', 'Canadw','CANADA','Canda','Csnada','Canad','Canadá',]
clean_reduced_df=clean_reduced_df.replace(mvp_ca, "Canada")

In [40]:
mvp_nz=['New Zealand ','New Zealand Aotearoa', 'New zealand','NZ',' New Zealand','new zealand','Aotearoa New Zealand','New Zealand',]
clean_reduced_df=clean_reduced_df.replace(mvp_nz, "New_Zealand")  

In [41]:
mvp_aus=['Australia ','Australia','australia','Australi','Australian ',]
clean_reduced_df=clean_reduced_df.replace(mvp_aus, "Australia")

In [42]:
mvp_nl=['The Netherlands','netherlands','Netherlands ','The Netherlands ','NL','the Netherlands', 'Nederland','The netherlands','the netherlands',]
clean_reduced_df=clean_reduced_df.replace(mvp_nl, "Netherlands")

In [43]:
mvp_esp=['Spain','Spain ','Catalonia','spain',]
clean_reduced_df=clean_reduced_df.replace(mvp_esp, "Spain")

In [44]:
mvp_ie=['Ireland','Ireland ','ireland',]
clean_reduced_df=clean_reduced_df.replace(mvp_ie, "Ireland")

In [45]:
# Creating Other for non defined countries
mvp_other=['Contracts',"We don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work, so I got a raise because a 3rd party assessment showed I was paid too little for the area we were located",'Global','Currently finance','UXZ','$2,175.84/year is deducted for benefits','IS','Remote','I.S.','bonus based on meeting yearly goals set w/ my supervisor',
       'International ',"I earn commission on sales. If I meet quota, I'm guaranteed another 16k min. Last year i earned an additional 27k. It's not uncommon for people in my space to earn 100k+ after commission. ",'Remote (philippines)','I was brought in on this salary to help with the EHR and very quickly was promoted to current position but compensation was not altered. ','n/a (remote from wherever I want)','From New Zealand but on projects across APAC', 'Y', 'europe',
       'UK, but for globally fully remote company',"USA, but for foreign gov't",'Policy','ss', 'dbfemf','LOUTRELAND','ff','Other','Africa',]
clean_reduced_df=clean_reduced_df.replace(mvp_other, "Other")

In [46]:
mvp_ger=['Germany','Germany ','germany','Company in Germany. I work from Pakistan.',]
clean_reduced_df=clean_reduced_df.replace(mvp_ger, "Germany")

In [47]:
mvp_jpn=['Japan', 'Japan ','japan',]
clean_reduced_df=clean_reduced_df.replace(mvp_jpn, "Japan")

In [48]:
mvp_ch=['Switzerland','SWITZERLAND','Switzerland ','switzerland',]
clean_reduced_df=clean_reduced_df.replace(mvp_ch, "Switzerland")

In [49]:
mvp_ind=['India','INDIA','ibdia',]
clean_reduced_df=clean_reduced_df.replace(mvp_ind, "India")

In [50]:
mvp_fr=['France','FRANCE','France ','france',]
clean_reduced_df=clean_reduced_df.replace(mvp_fr, "France")

In [51]:
mvp_den=['Denmark','Denmark ','Danmark','denmark','Denmark ',]
clean_reduced_df=clean_reduced_df.replace(mvp_den, "Denmark")

In [52]:
mvp_rsa=['South Africa ','South Africa','South africa',]
clean_reduced_df=clean_reduced_df.replace(mvp_rsa, "South Africa")

In [53]:
mvp_uae=['United Arab Emirates ','I work for a UAE-based organization, though I am personally in the US.',]
clean_reduced_df=clean_reduced_df.replace(mvp_uae, "UAE")   

In [54]:
mvp_mm=['Myanmar', 'Burma',]
clean_reduced_df=clean_reduced_df.replace(mvp_mm, "Myanmar") 

In [55]:
mvp_hkg=['Hong Kong','Hong Kong ','hong konh',]
clean_reduced_df=clean_reduced_df.replace(mvp_hkg, "Hong Kong")

In [56]:
mvp_chn=['China','Mainland China',]
clean_reduced_df=clean_reduced_df.replace(mvp_chn, "China") 

In [57]:
mvp_cz=['Czech republic','Czechia','czech republic','Czech Republic ','Czech Republic',]
clean_reduced_df=clean_reduced_df.replace(mvp_cz, "Czech Republic") 

In [58]:
mvp_ua=['U.A.','UA','Ukraine ',]
clean_reduced_df=clean_reduced_df.replace(mvp_ua, "Ukraine")

In [59]:
mvp_cro=['Croatia ','croatia',]
clean_reduced_df=clean_reduced_df.replace(mvp_cro, "Croatia")

In [60]:
mvp_sg=['Singapore','Singapore ','singapore',]
clean_reduced_df=clean_reduced_df.replace(mvp_sg, "Singapore")

In [61]:
mvp_th=['Thailand','ARGENTINA BUT MY ORG IS IN THAILAND',]
clean_reduced_df=clean_reduced_df.replace(mvp_th, "Thailand")

In [62]:
mvp_se=['Sweden','Sweden ',]
clean_reduced_df=clean_reduced_df.replace(mvp_se, "Sweden")

In [63]:
mvp_mx=['Mexico ','Mexico','México',]
clean_reduced_df=clean_reduced_df.replace(mvp_mx, "Mexico")

In [64]:
mvp_bra=['Brazil','Brazil ','Brasil',]
clean_reduced_df=clean_reduced_df.replace(mvp_bra, "Brazil")

In [65]:
mvp_lux=['Luxembourg','Luxemburg',]
clean_reduced_df=clean_reduced_df.replace(mvp_lux, "Luxemburg")

In [66]:
mvp_lk=['Sri lanka','Sri Lanka',]
clean_reduced_df=clean_reduced_df.replace(mvp_lk, "Sri Lanka")

In [67]:
mvp_fi=['Finland','finland',]
clean_reduced_df=clean_reduced_df.replace(mvp_fi, "Finland")

In [68]:
mvp_ita=['Italy (South)','Italy',]
clean_reduced_df=clean_reduced_df.replace(mvp_ita, "Italy")

In [69]:
mvp_pak=['Pakistan','pakistan',]
clean_reduced_df=clean_reduced_df.replace(mvp_pak, "Pakistan")

In [70]:
mvp_kr=['South Korea','South Korea ',]
clean_reduced_df=clean_reduced_df.replace(mvp_kr, "South Korea")

In [71]:
mvp_bd=['Bangladesh ','Bangladesh',]
clean_reduced_df=clean_reduced_df.replace(mvp_bd, "Bangladesh")

In [72]:
mvp_ph=['philippines',]
clean_reduced_df=clean_reduced_df.replace(mvp_ph, "Philippines")

In [73]:
mvp_civ=["Cote d'Ivoire",]
clean_reduced_df=clean_reduced_df.replace(mvp_civ, "Ivory Coast")

In [74]:
mvp_bel=['Belgium ',]
clean_reduced_df=clean_reduced_df.replace(mvp_bel, "Belgium")

In [75]:
mvp_bhs=['The Bahamas ',]
clean_reduced_df=clean_reduced_df.replace(mvp_bhs, "The Bahamas")

In [76]:
mvp_no=['Norway ',]
clean_reduced_df=clean_reduced_df.replace(mvp_no, "Norway")

In [77]:
mvp_my=['Malaysia ',]
clean_reduced_df=clean_reduced_df.replace(mvp_my, "Malaysia")

In [78]:
mvp_ru=['Russia ',]
clean_reduced_df=clean_reduced_df.replace(mvp_ru, "Russia")

In [79]:
mvp_pt=['Portugal ',]
clean_reduced_df=clean_reduced_df.replace(mvp_pt, "Portugal")

In [80]:
mvp_pl=['Poland ',]
clean_reduced_df=clean_reduced_df.replace(mvp_pl, "Poland")

In [81]:
mvp_pa=['Panamá',]
clean_reduced_df=clean_reduced_df.replace(mvp_pa, "Panama")

In [82]:
mvp_ng=['NIGERIA', 'Nigeria ',]
clean_reduced_df=clean_reduced_df.replace(mvp_ng, "Nigeria")

In [83]:
# Cleaning Annual_salary feature
clean_reduced_df.head()

Industry                                 Job_title  \
0   Education (Higher Education)        Research and Instruction Librarian   
1              Computing or Tech  Change & Internal Communications Manager   
2  Accounting, Banking & Finance                      Marketing Specialist   
3                     Nonprofits                           Program Manager   
4  Accounting, Banking & Finance                        Accounting Manager   

  Annual_salary Currency Country_of_work Years_of_experience_in_your_field  \
0        55,000      USD            U.S.                         5-7 years   
1        54,600      GBP            U.K.                         5-7 years   
2        34,000      USD            U.S.                       2 - 4 years   
3        62,000      USD            U.S.                         5-7 years   
4        60,000      USD            U.S.                         5-7 years   

  Level_of_education  Average_age Gender_category  Average_Years_of_experience  
0    Master's degree         29.0          Female                          6.0  
1     College degree         29.0      Non-binary                          9.0  
2     College degree         29.0          Female                          3.0  
3     College degree         29.0          Female                          9.0  
4     College degree         29.0          Female                          9.0

In [84]:
clean_reduced_df["Annual_salary"].unique()

array(['55,000', '54,600', '34,000', ..., '7000', '37741', '53060'],
      dtype=object)

In [85]:
# Removing commas from the features and converting to numeric values just for the Annual_salary feature
clean_reduced_df["Annual_salary"] = clean_reduced_df["Annual_salary"].replace({',': ''}, regex=True)

In [86]:
# Checking what data type is Annual Salary feature now
clean_reduced_df.dtypes

Industry                              object
Job_title                             object
Annual_salary                         object
Currency                              object
Country_of_work                       object
Years_of_experience_in_your_field     object
Level_of_education                    object
Average_age                          float64
Gender_category                       object
Average_Years_of_experience          float64
dtype: object

In [87]:
clean_reduced_df.head()

Industry                                 Job_title  \
0   Education (Higher Education)        Research and Instruction Librarian   
1              Computing or Tech  Change & Internal Communications Manager   
2  Accounting, Banking & Finance                      Marketing Specialist   
3                     Nonprofits                           Program Manager   
4  Accounting, Banking & Finance                        Accounting Manager   

  Annual_salary Currency Country_of_work Years_of_experience_in_your_field  \
0         55000      USD            U.S.                         5-7 years   
1         54600      GBP            U.K.                         5-7 years   
2         34000      USD            U.S.                       2 - 4 years   
3         62000      USD            U.S.                         5-7 years   
4         60000      USD            U.S.                         5-7 years   

  Level_of_education  Average_age Gender_category  Average_Years_of_experience  
0    Master's degree         29.0          Female                          6.0  
1     College degree         29.0      Non-binary                          9.0  
2     College degree         29.0          Female                          3.0  
3     College degree         29.0          Female                          9.0  
4     College degree         29.0          Female                          9.0

In [88]:
# Replacing all of the blanks (nan) with zero values
clean_reduced_df = clean_reduced_df.fillna(0)

In [89]:
# Converting Annual Salary feature into numerical
# Reference: Stack Overflow https://stackoverflow.com/questions/48094854/pandas-convert-data-type-from-object-to-float
clean_reduced_df["Annual_salary"] = clean_reduced_df["Annual_salary"].astype(float)

In [90]:
# Checking if the data type has changed from object to float64 in the Annual Salary feature
clean_reduced_df.dtypes

Industry                              object
Job_title                             object
Annual_salary                        float64
Currency                              object
Country_of_work                       object
Years_of_experience_in_your_field     object
Level_of_education                    object
Average_age                          float64
Gender_category                       object
Average_Years_of_experience          float64
dtype: object

In [91]:
# Creating a new dictionary for Currency feature
rename_map = {'AUD/NZD':'AUD',
              '0':'Other',}

In [92]:
# Reference: Stack Overflow, https://stackoverflow.com/questions/46432315/pandas-replace-column-values-by-dictionary-keys-if-they-are-in-dictionary-valu
clean_reduced_df["Currency"] = clean_reduced_df["Currency"].replace(rename_map)

In [93]:
print(clean_reduced_df["Currency"])

0        USD
1        GBP
2        USD
3        USD
4        USD
        ... 
28080    USD
28081    USD
28082    EUR
28083    USD
28084    USD
Name: Currency, Length: 28085, dtype: object


In [94]:
# Converting all of the currencies into EUR, using Central Bank of Irelands daily FX rates, latest available date is 25 October 2024
# Reference Central Bank of Ireland https://www.centralbank.ie/statistics/interest-rates-exchange-rates/exchange-rates
# Converting to one currency as common practice in finances to analyse discrepancies
exchange_rates = {
    "USD": 1.0915,  # 1 USD = 1.0915 EUR
    "GBP": 0.83358,  # 1 GBP = 0.83358 EUR
    "EUR": 1.0,   # EUR is already in EUR
    "JPY": 164.45, # 1 JPY = 164.45 EUR
    "CAD": 1.4989, # 1 CAD = 1.4989 EUR
    "AUD": 1.6311, # 1 AUD = 1.6311 EUR
    "Other": 1.0, # No need to convert
    "ZAR": 19.0627, # 1 ZAR = 19.0627 EUR
    "SEK": 11.4475, # 1 SEK = 11.4475 EUR
    "HKD": 8.4110 # 1 HKD = 8.4110 EUR
}

In [95]:
# Referencing Practical Business Phyton; https://pbpython.com/currency-cleanup.html by Chris Moffitt 3rd of November 2019
clean_reduced_df["Annual_Salary_in_EUR"] = clean_reduced_df.apply(lambda row: row["Annual_salary"] * exchange_rates.get(row["Currency"], 1), axis=1)

In [96]:
print(clean_reduced_df["Annual_Salary_in_EUR"])

0          60032.500
1          45513.468
2          37111.000
3          67673.000
4          65490.000
            ...     
28080     127705.500
28081      76405.000
28082      28600.000
28083      61298.640
28084    2183000.000
Name: Annual_Salary_in_EUR, Length: 28085, dtype: float64


In [97]:
# Checking if there are missing values
print(clean_reduced_df["Annual_Salary_in_EUR"].isnull().sum())

0


In [98]:
print(clean_reduced_df)

                                   Industry  \
0              Education (Higher Education)   
1                         Computing or Tech   
2             Accounting, Banking & Finance   
3                                Nonprofits   
4             Accounting, Banking & Finance   
...                                     ...   
28080         Accounting, Banking & Finance   
28081         Education (Primary/Secondary)   
28082  Government and Public Administration   
28083                     Computing or Tech   
28084                       Social networks   

                                      Job_title  Annual_salary Currency  \
0            Research and Instruction Librarian        55000.0      USD   
1      Change & Internal Communications Manager        54600.0      GBP   
2                          Marketing Specialist        34000.0      USD   
3                               Program Manager        62000.0      USD   
4                            Accounting Manager        60000

DUPLICATES

In [100]:
clean_reduced_df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
28080    False
28081    False
28082    False
28083    False
28084    False
Length: 28085, dtype: bool

In [101]:
clean_reduced_df.duplicated().sum()

213

In [102]:
# Duplicated values which is normal in for a database of this size and queries in survey
clean_reduced_df.duplicated("Annual_Salary_in_EUR").sum()

23452

OUTLIERS